# Channel Estimation

In [2]:
using DSP
using FFTW

In [3]:
# Some Math tools
# Inverse Fourier Transform function
Fᴵ = ifft 

# Convolution infix operation
function ⊗(a, b) 
    return conv(a,b)
end
# Transpose postfix operation
struct Transposer end
const ᵀ = Transposer() #typed \^T<TAB>
Base.:(*)(x, ::Transposer) = transpose(x)

In [4]:
include("rxsignal_withchannelandfreqoff.jl");

In [5]:
# RX Received Signal 3
Ŝᵣₓ³ = rxs3;

In [6]:
include("pss2.jl");

In [9]:
# PSS₂ signals and such
PSS₂ = pss_2;
PSS₂ᵀ = transpose.(PSS₂); # Hessian Transpose ?

In [10]:
𝑆ₚₛₛ² = Fᴵ(PSS₂ᵀ); # S Slanted (fourier transform) in time domain

In [11]:
function freq_offset_est(signal, pss_1, Nf)
    # Frequency offset estimator

    DELTA_F = 10.0

    Fs = 61.44e6
    f_min = -7500
    f_max = 7500

    m = f_min:DELTA_F:f_max
    Y = zeros(Float64, length(m))
    L = length(pss_1)
    t = 0:(1/Fs):((L-1)/Fs)

    signal_part = signal[Nf:Nf+L-1]
    convolve = conj.(pss_1) .* signal_part'

    for j = 1:length(m)
        signal_offset = sum(exp.(-2*pi*1.0im*m[j].*t) .* convolve)
        value = abs(signal_offset)^2
        Y[j] += value
    end

    (A_fo, fo) = argmax(Y)
    # println("Detected offset = 150Mhz", real(m2_chan))

    println("m ", length(m), ", A_fo ", A_fo, ", fo ", fo)

    plot(1:length(m), Y, ".", label="Frequency offset")
end


freq_offset_est (generic function with 1 method)

In [13]:
S̅ₚₛₛ² = reverse(𝑆ₚₛₛ²); # reverse
S̅ₚₛₛ²ᴴ = conj(S̅ₚₛₛ²); # conjugate 
argmax = findmax

findmax (generic function with 8 methods)

In [14]:
m̂₂ = 10 * log10.(abs.( Ŝᵣₓ³ ⊗ S̅ₚₛₛ²ᴴ )); ; size( m̂₂ )

(618494, 1)

In [15]:
# Find maximum value and its index
Ĉᵩ², N̂ᵩ² = argmax(m̂₂) #; m2_chan[NF2_chan] or direct m2_chan[(6628)...] 

(69.86256301907788, CartesianIndex(6628, 1))

In [17]:
# Grab the function Profs Frequency Offset with his values
freq_offset_est(Ŝᵣₓ³, 𝑆ₚₛₛ², Ĉᵩ² - length(𝑆ₚₛₛ²) + 1)
# Detected offset: 150 Hz

LoadError: BoundsError: attempt to access 616447×1 Matrix{ComplexF64} at index [-1977.137436980922:1.0:69.86256301907792]